<a href="https://colab.research.google.com/github/simulate111/Introduction-to-Human-Language-Technology/blob/main/Excercise%20task%2010%3A%20word%20embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet sentence-splitter regex
from sentence_splitter import SentenceSplitter
import requests
import regex
!pip install gensim
from gensim.models import Word2Vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.1 MB/s eta 0:00:00


In [2]:
# Download the English dataset
data_url = "http://dl.turkunlp.org/TKO_7095_2023/enwiki-20220301-sample.txt"
response = requests.get(data_url)
data = response.text

In [3]:
# Split to paragraphs
paragraphs = data.split('\n')
# As the udpipe tooks few hours, I select sentence splitter library.
splitter = SentenceSplitter(language='en')

In [4]:
%%time
sentences = [s for p in paragraphs[:1000000] for s in splitter.split(p)]

CPU times: user 12min 18s, sys: 3.07 s, total: 12min 21s
Wall time: 12min 25s


In [5]:
%%time
TOKENIZE_RE = regex.compile(r'([[:alnum:]]+|\S)')

CPU times: user 664 µs, sys: 0 ns, total: 664 µs
Wall time: 627 µs


In [6]:
%%time
tokenized_sentences = [TOKENIZE_RE.findall(s.lower()) for s in sentences]

CPU times: user 1min 10s, sys: 3.32 s, total: 1min 13s
Wall time: 1min 13s


In [7]:
# Print some first rows
for i in range(10):
    print(tokenized_sentences[i])

['incumbent', 'cm', 'mayawati', 'began', 'her', 'campaign', 'on', '27', 'january', 'at', 'a', 'rally', 'in', 'bijnor', '.']
['on', '15', 'january', ',', 'she', 'released', 'the', 'bsp', "'", 's', 'list', 'of', 'candidates', 'for', 'all', 'the', '403', 'constituencies', '.']
['the', 'list', 'included', '88', 'candidates', 'belonging', 'to', 'scs', ',', '113', 'from', 'obcs', ',', '85', 'religious', 'minorities', 'and', '117', 'upper', 'castes', ',', 'out', 'of', 'which', '74', 'are', 'brahmins', '.']
['as', 'a', 'member', 'of', 'the', 'oss', 'research', 'and', 'analysis', 'division', ',', 'wheeler', 'had', 'government', 'security', 'clearance', 'to', 'received', 'secret', 'and', 'confidential', '"', 'ditto', '"', 'copies', 'of', 'monthly', 'and', 'semi', '-', 'monthly', 'reports', 'of', 'political', 'developments', 'throughout', 'the', 'world', '.']
['wheeler', 'is', 'alleged', 'to', 'have', 'passed', 'these', 'reports', 'as', 'well', 'as', 'handwritten', 'and', 'typewritten', 'material

In [8]:
# Word2Vec
model = Word2Vec(sentences=tokenized_sentences)

In [9]:
# FInd similar Words to:
words = ["good", "bad", "dog", "cat", "king", "queen"]
similar_words = {}
for word in words:
    similar_words[word] = model.wv.most_similar(word)

In [10]:
for word, similar in similar_words.items():
    print(word, similar)

good [('bad', 0.7764500379562378), ('decent', 0.7107490301132202), ('tough', 0.6820545196533203), ('perfect', 0.6808227896690369), ('genuine', 0.6517764329910278), ('sensible', 0.6490606665611267), ('happy', 0.6432491540908813), ('little', 0.6407383680343628), ('quiet', 0.6328219175338745), ('fun', 0.6302210688591003)]
bad [('good', 0.7764500975608826), ('terrible', 0.6831875443458557), ('funny', 0.6592331528663635), ('horrible', 0.6534147262573242), ('tough', 0.6473285555839539), ('nasty', 0.6356762647628784), ('happy', 0.63502436876297), ('crazy', 0.6306237578392029), ('cute', 0.6289855241775513), ('poor', 0.6222598552703857)]
dog [('cat', 0.8436651229858398), ('puppy', 0.7868298292160034), ('goat', 0.7807270288467407), ('rabbit', 0.7669864892959595), ('boar', 0.7664580345153809), ('dogs', 0.7599594593048096), ('pig', 0.7445487976074219), ('monkey', 0.7392084002494812), ('cow', 0.7383224368095398), ('pet', 0.7315573692321777)]
cat [('rabbit', 0.8864553570747375), ('monkey', 0.8563873